# Kaggle notebook

In [ ]:
# Install required dependencies with PyTorch development headers
!pip install huggingface_hub transformers accelerate pybind11
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!apt-get update && apt-get install -y cmake build-essential
!pip install pybind11

In [ ]:
# Copy source files to working directory
import shutil
import os
import pybind11
import torch

# Create directories
os.makedirs('/kaggle/working/Diffusion.cu/cuda_kernel', exist_ok=True)
os.makedirs('/kaggle/working/Diffusion.cu/diffusion_model', exist_ok=True)

# Copy CUDA kernel files to be able to execute & write access
shutil.copy2('/kaggle/input/diff12/Diffusion.cu/cuda_kernel/flash_atten.cu', '/kaggle/working/Diffusion.cu/cuda_kernel/')
shutil.copy2('/kaggle/input/diff12/Diffusion.cu/cuda_kernel/groupnorm.cu', '/kaggle/working/Diffusion.cu/cuda_kernel/')
shutil.copy2('/kaggle/input/diff12/Diffusion.cu/cuda_kernel/conv2d_k3.cu', '/kaggle/working/Diffusion.cu/cuda_kernel/')

# Copy CMakeLists.txt
shutil.copy2('/kaggle/input/diff12/Diffusion.cu/CMakeLists.txt', '/kaggle/working/Diffusion.cu/')

# Create build directory and compile
os.makedirs('/kaggle/working/Diffusion.cu/build', exist_ok=True)
%cd /kaggle/working/Diffusion.cu/build
# now we are in build dir

pybind11_dir = pybind11.get_cmake_dir()
torch_dir = torch.utils.cmake_prefix_path

print(f"Pybind11 CMake directory: {pybind11_dir}")
print(f"Torch CMake directory: {torch_dir}")

# Configure and build with both pybind11 and torch paths
!cmake .. -Dpybind11_DIR={pybind11_dir} -DTorch_DIR={torch_dir}/Torch -DCMAKE_BUILD_TYPE=Release
!make -j4

# Check if compilation was successful
print("Compilation output:")
!ls -la *.so

# Add the compiled modules to Python path
import sys
sys.path.insert(0, '/kaggle/working/Diffusion.cu/build')

In [ ]:
# Copy all Python files from diffusion_model
import glob

python_files = glob.glob('/kaggle/input/diff12/Diffusion.cu/diffusion_model/*.py')
for file_path in python_files:
    shutil.copy2(file_path, '/kaggle/working/Diffusion.cu/diffusion_model/')

# Add both directories to Python path
sys.path.insert(0, '/kaggle/working/Diffusion.cu/diffusion_model')
sys.path.insert(0, '/kaggle/working/Diffusion.cu/build')

In [ ]:
# First, let's check what modules are actually available
import os
print("Available compiled modules:")
for file in os.listdir('/kaggle/working/Diffusion.cu/build'):
    if file.endswith('.so'):
        print(f"  {file}")

In [ ]:
import sys
import os
import torch
from PIL import Image
from pathlib import Path
from transformers import CLIPTokenizer
from huggingface_hub import hf_hub_download

# Add paths
sys.path.insert(0, '/kaggle/working/Diffusion.cu/diffusion_model')
sys.path.insert(0, '/kaggle/working/Diffusion.cu/build')

try:
    import model_loader
    import pipeline
    print("✓ Successfully imported model_loader and pipeline")
except ImportError as e:
    print(f"✗ Failed to import custom modules: {e}")
    # Try alternative import
    try:
        from diffusion_model import model_loader, pipeline
        print("✓ Successfully imported from diffusion_model")
    except ImportError as e2:
        print(f"✗ Alternative import also failed: {e2}")

In [ ]:
# GPU Configuration for Kaggle
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
ALLOW_CUDA = True

if torch.cuda.is_available() and ALLOW_CUDA:
    DEVICE = "cuda"
    print(f"Using GPU: {torch.cuda.get_device_name()}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    DEVICE = "cpu"
    print("Using CPU")

print(f"Using device: {DEVICE}")

In [ ]:
# Create working directories in Kaggle
os.makedirs('/kaggle/working/data', exist_ok=True)
os.makedirs('/kaggle/working/images', exist_ok=True)

In [ ]:
# Download tokenizer files from Hugging Face
print("Downloading tokenizer files...")
try:
    vocab_path = hf_hub_download(
        repo_id="sd-legacy/stable-diffusion-v1-5",
        filename="tokenizer/vocab.json",
        cache_dir="/kaggle/working/data",
        force_download=True
    )
    merges_path = hf_hub_download(
        repo_id="sd-legacy/stable-diffusion-v1-5", 
        filename="tokenizer/merges.txt",
        cache_dir="/kaggle/working/data",
        force_download=True
    )
    print("Tokenizer files downloaded successfully")
except Exception as e:
    print(f"Error downloading tokenizer files: {e}")
    # Fallback: try to use paths from your dataset if they exist
    vocab_path = "/kaggle/working/data/vocab.json"
    merges_path = "/kaggle/working/data/merges.txt"

In [ ]:
# Download model weights
print("Downloading model weights...")
try:
    model_path = hf_hub_download(
        repo_id="sd-legacy/stable-diffusion-v1-5",
        filename="v1-5-pruned-emaonly.ckpt",
        cache_dir="/kaggle/working/data",
        force_download=True
    )
    print("Model weights downloaded successfully")
except Exception as e:
    print(f"Error downloading model weights: {e}")
    # If download fails, you might need to manually upload the model to Kaggle dataset
    model_path = "/kaggle/working/data/v1-5-pruned-emaonly.ckpt"
    print("Please ensure model weights are available at the above path")

In [ ]:
# Initialize tokenizer
try:
    tokenizer = CLIPTokenizer(vocab_path, merges_file=merges_path)
    print("Tokenizer initialized successfully")
except Exception as e:
    print(f"Error initializing tokenizer: {e}")
    # Try alternative path
    try:
        tokenizer = CLIPTokenizer("/kaggle/working/data/tokenizer/vocab.json", 
                                merges_file="/kaggle/working/data/tokenizer/merges.txt")
    except:
        print("Failed to initialize tokenizer. Please check file paths.")

In [ ]:
# Load models
print("Loading models...")
torch.cuda.empty_cache()

models = model_loader.preload_models_from_standard_weights(model_path, DEVICE)
print("Models loaded successfully")

# Clear GPU cache if using CUDA
if DEVICE == "cuda":
    torch.cuda.empty_cache()

In [ ]:
# Only proceed if models were loaded successfully
if models is not None:
    # TEXT TO IMAGE
    prompt = "A cat stretching on the floor, highly detailed, ultra sharp, cinematic, 100mm lens, 8k resolution."
    uncond_prompt = ""  # Also known as negative prompt
    do_cfg = True
    cfg_scale = 8  # min: 1, max: 14

    # IMAGE TO IMAGE (disabled by default)
    input_image = None
    # Uncomment for image-to-image
    # image_path = "/kaggle/working/images/input.jpg"
    # input_image = Image.open(image_path)
    strength = 0.9

    # SAMPLER
    sampler = "ddpm"
    num_inference_steps = 30  # Reduced for faster generation on Kaggle
    seed = 42

    # Set seed for reproducibility
    torch.manual_seed(seed)
    if DEVICE == "cuda":
        torch.cuda.manual_seed(seed)

In [ ]:
print(f"Starting image generation on {DEVICE}...")

# Time the generation
import time
start_time = time.time()

if models is not None:  # ✅ This is line 41
    try:
        output_image = pipeline.generate(
            prompt=prompt,
            uncond_prompt=uncond_prompt,
            input_image=input_image,
            strength=strength,
            do_cfg=do_cfg,
            cfg_scale=cfg_scale,
            sampler_name=sampler,
            n_inference_steps=num_inference_steps,
            seed=seed,
            models=models,
            device=DEVICE,
            idle_device="cuda" if DEVICE == "cuda" else "cpu",
            tokenizer=tokenizer,
        )

        end_time = time.time()
        print(f"Image generation completed in {end_time - start_time:.2f} seconds")

        # Save and display the image
        output_path = "/kaggle/working/generated_image.png"
        Image.fromarray(output_image).save(output_path)
        print(f"Image saved to: {output_path}")
        
        # Display the image
        display(Image.fromarray(output_image))
        
    except Exception as e:
        print(f"Error during image generation: {e}")
        print("This might be due to:")
        print("1. Memory issues - try reducing num_inference_steps")
        print("2. Model compatibility issues")
        print("3. Missing dependencies")
else:  # ✅ This should be at the same indentation level as the 'if' statement
    print("Cannot generate image - models failed to load")